In [251]:
import re

In [252]:
class CPU:
    name:int    # sec
    usr:float   # %
    sys:float   # %
    idle:float  # %

class TOP:
    time        :str    # sec
    cpus        :list   # # of CPU


In [253]:
def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [254]:
def parseCPU(cpu:str):
    raw = CPU()
    raw.name = 'c' + re.findall(r'%Cpu([\d])\s', cpu)[0]
    raw.usr = re.findall(r'([\d.]+)\sus,', cpu)[0]
    raw.sys = re.findall(r'([\d.]+)\ssy,', cpu)[0]
    raw.idle = re.findall(r'([\d.]+)\sid,', cpu)[0]
    return raw

In [255]:
def parseTop(top:list):
    raw = TOP()
    cpus = []
    try:
        for line in top:
            if not line : continue
            if 'top' in line:
                time = re.findall(r'top - ([\d\:]+)\s', line)[0]
                raw.time = get_sec(time)
                continue
            # print('line : ', line)
            cpus.append(parseCPU(line))
            # raw.cpus.append(cpu)
        
        raw.cpus = cpus
        return raw
    except:
        # print(top)
        print('except: ', raw.__dict__)

In [256]:
def parseTrace(filename):
    infile = open(filename, 'r')

    lines=[]
    result=[]
    for line in infile.readlines():
        if not line : continue
        # print(idx, line)
        if('top' in line):
            if(len(lines) > 1):
                result.append(parseTop(lines))
                lines.clear()
        lines.append(line)
        
    infile.close()
    outfile = open(filename + '.parse', 'w')

    # Header
    outfile.write('time,')
    for cpu in result[0].cpus:
        outfile.write(cpu.name + '_usr,')
        outfile.write(cpu.name + '_sys,')
        outfile.write(cpu.name + '_idle,')
    outfile.write('\n')

    # Data
    startTime = result[0].time
    for stamp in result:
        outfile.write(str(stamp.time - startTime) + ',')
        for cpu in stamp.cpus:
            outfile.write(cpu.usr + ',')
            outfile.write(cpu.sys + ',')
            outfile.write(cpu.idle + ',')
        outfile.write('\n')
    # for cpu in result[0].cpus:
    #     outfile.write(f'{cpu.name}_usr,{cpu.name}_sys,{cpu.name}_idle\n')

    # for item in result:
    #     # outfile.write(item.)
    #     outfile.write(','.join(item.__dict__.values()) + '\n')
    
    outfile.close()

    return outfile.name

In [257]:
filename = './raw/top.txt'
parsedFile = parseTrace(filename)